## Notebook for execution of training and evaluation of the model.

In [7]:
import pandas as pd

df = pd.read_csv('../data/pre-processed.csv')
print(df.iloc[-1])


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load preprocessed data
df = pd.read_csv('../data/raw/pre-processed.csv')  

# Confirm class balance
print(df['label'].value_counts())

# Split: 70% train, 15% val, 15% test
df_train, df_temp = train_test_split(
    df, test_size=0.30, stratify=df['label'], random_state=42
)
df_val, df_test = train_test_split(
    df_temp, test_size=0.5, stratify=df_temp['label'], random_state=42
)

print("Train size:", len(df_train))
print("Val size:", len(df_val))
print("Test size:", len(df_test))

# Save splits
df_train.to_csv('../data/processed/train.csv', index=False)
df_val.to_csv('../data/processed/val.csv', index=False)
df_test.to_csv('../data/processed/test.csv', index=False)


label
fake    3600
true    3600
Name: count, dtype: int64
Train size: 5040
Val size: 1080
Test size: 1080


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Load splits
df_train = pd.read_csv('../data/processed/train.csv')
df_val = pd.read_csv('../data/processed/val.csv')
df_test = pd.read_csv('../data/processed/test.csv')

# Prepare features and labels
X_train, y_train = df_train['text'], df_train['label']
X_val, y_val = df_val['text'], df_val['label']
X_test, y_test = df_test['text'], df_test['label']

# TF-IDF Vectorizer (fit only on train!)
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_vec = tfidf.fit_transform(X_train)
X_val_vec = tfidf.transform(X_val)
X_test_vec = tfidf.transform(X_test)

# Logistic Regression
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train_vec, y_train)

# Validation metrics
val_pred = clf.predict(X_val_vec)
print("Validation Metrics:")
print(classification_report(y_val, val_pred))

# Test metrics
test_pred = clf.predict(X_test_vec)
print("Test Metrics:")
print(classification_report(y_test, test_pred))
print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, test_pred))


KeyError: 'text'

In [ ]:
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np
X = df["preprocessed_news"]
Y = df["label"]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
def get_embeddings(text, model, tokenizer):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[0, 0, :]
    return cls_embedding

embeddings = []
for i in tqdm(range(len(X))):
    e = get_embeddings(X.iloc[i], model, tokenizer)
    embeddings.append(e.detach().numpy())
embeddings = np.array(embeddings)

  0%|          | 0/225 [00:00<?, ?it/s]

  1%|          | 2/225 [00:19<35:40,  9.60s/it]


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

SyntaxError: invalid non-printable character U+00A0 (3745882828.py, line 9)